In [1]:
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'assignment_2'))
	print(os.getcwd())
except:
	pass

/Users/nwoodle/Developer/CSE258_recommender_system/assignment_2


# Assignment 2 

In [2]:
from preprocessing import *

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/nwoodle/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [34]:
dataset = data_preprocessing(timethresh=2)

In [35]:
dataset["avg_agi"].replace('', np.nan, inplace=True)
dataset.dropna(subset=["avg_agi"], inplace=True)

In [36]:
train, test, valid = train_test_validation(dataset)

In [37]:
from sklearn import linear_model

In [38]:
model = linear_model.LogisticRegression(C = 0.005, class_weight='balanced')

In [39]:
train_data, train_label = data_label(train)

In [40]:
valid_data, valid_label = data_label(valid)
test_data, test_label = data_label(test)

In [41]:
model.fit(train_data, train_label)

/Users/nwoodle/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nwoodle/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=0.005, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [42]:
model.score(valid_data, valid_label)

0.7382228265216348

In [43]:
model.score(test_data, test_label)

0.7283634618461047

In [44]:
model.score(train_data, train_label)

0.7359501694678011

In [45]:
import xgboost as xgb

In [46]:
D_train = xgb.DMatrix(train_data, label=train_label)
D_valid = xgb.DMatrix(valid_data, label=valid_label)

In [53]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 2} 

steps = 50  # The number of training iterations
model = xgb.train(param, D_train, steps)

In [54]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_valid)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(valid_label, best_preds, average='macro')))
print("Recall = {}".format(recall_score(valid_label, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(valid_label, best_preds)))

Precision = 0.792713511067106
Recall = 0.7237184205880598
Accuracy = 0.7505398704310965


In [55]:
preds = model.predict(D_train)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(train_label, best_preds, average='macro')))
print("Recall = {}".format(recall_score(train_label, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(train_label, best_preds)))

Precision = 0.8035869661104176
Recall = 0.7313209185344554
Accuracy = 0.7587258420900029
